# Erreichbarkeit des U Bahn Netzes

In dieser ersten realistischen Aufgabenstellung wollen wir ermitteln, wie viel % der Wiener
Bevölkerung pro Bezirk das U Bahn Netz vom Wohnort aus leicht erreichen kann. Dafür nehmen wir alle Adressen
und prüfen, ob sich eine U Bahn Station im Umkreis von 1 km Entfernung befindet.

## Laden der Rohndaten

### Die Adressen

Das BEV (Bundesamt für Eich- und Vermessungswesen) bietet das öffentlich zugängliche
[Adressregister](https://www.bev.gv.at/portal/page?_pageid=713,2601271&_dad=portal&_schema=PORTAL)
zum Download an. Es besteht aus mehreren CSV Dateien. Wir verwenden die Datei *ADRESSE.csv* aus
dem Archiv *Adresse Relationale Tabellen - Stichtagsdaten 03.04.2022.zip*. Diese Datei liegt bz2
komprimiert im Repository bereit.

Die geografischen Angaben sind nicht mit Längen- und Breitengrad codiert, sondern mit einem eigenen
Koordinatensystem aus der Vermessung. Diese Koordinatensysteme projizieren einen begrenzten Bereich
auf 2D Koordinaten, damit Berechnungen schneller ablaufen. Deswegen verwenden wir das Paket
*pyproj*, um diese Koordinaten umzuwandeln. Es muss vorher installiert werden:

```
pip install pyproj
```

Das Laden der Datei samt Projektion wird in etwa 1 Minute (je nach Rechnergeschwindigkeit) dauern.

In [92]:
import pyproj                               # pip install pyproj
import pandas as pd
# Von epsg:31256 MGI / Austria GK East (https://epsg.io/31256)
# auf epsg:4326 (GPS)
transformer = pyproj.Transformer.from_crs("epsg:31256", "epsg:4326")
# ADRCD ist eindeutig und wir verwenden ihn als Index für den Dataframe.
addresses = pd.read_csv("ADRESSE.csv.bz2", sep=";", usecols=['ADRCD', 'GKZ', 'PLZ', 'RW', 'HW'])
addresses = addresses[addresses.GKZ == 90001]  # Nur Wien laden
# Die Koordinaten umrechnen und dafür die Spalten LAT und LNG erstellen.
gps = addresses.apply(lambda a: transformer.transform(a.HW, a.RW), axis=1)
addresses[["LAT", "LNG"]] = gps.apply(pd.Series)
addresses.head()

,ADRCD,GKZ,PLZ,RW,HW,LAT,LNG
2318027,6860141,90001,1140,-2817.33,340755.89,48.205199,16.294231
2318028,6860142,90001,1140,-2796.39,340751.21,48.205158,16.294513
2318029,6879850,90001,1160,-109.61,341405.51,48.211049,16.330659
2318030,6879851,90001,1160,-190.49,341438.31,48.211344,16.329571
2318031,6879853,90001,1160,-201.17,341444.45,48.211399,16.329427


### Die Stationen

Wie in den Übungen brauchen wir wieder unsere Stationen der Wiener Linien. Diese laden wir wie in
den vorigen Beispielen aus dem Netz. Wir laden jedoch nur die Haltestellen von Wien und die Spalten,
die wir brauchen. Das spart Speicher und erleichtert den Umgang mit den Dataframes.

In [90]:
stations = pd.read_csv("https://data.wien.gv.at/csv/wienerlinien-ogd-haltestellen.csv",
    sep=";", encoding="utf-8",
    usecols=["HALTESTELLEN_ID", "NAME", "GEMEINDE_ID", "WGS84_LAT", "WGS84_LON"]) \
    .rename(columns={"WGS84_LAT": "LAT", "WGS84_LON": "LNG"})
stations = stations[stations.GEMEINDE_ID == 90001] \
    .set_index("HALTESTELLEN_ID", drop=False)

linien = pd.read_csv("https://data.wien.gv.at/csv/wienerlinien-ogd-linien.csv",
    sep=";", encoding="utf-8",
    usecols=["LINIEN_ID", "BEZEICHNUNG", "VERKEHRSMITTEL"]) \
    .set_index("LINIEN_ID", drop=False)

steige = pd.read_csv("https://data.wien.gv.at/csv/wienerlinien-ogd-steige.csv",
    sep=";", encoding="utf-8",
    usecols=["STEIG_ID", "FK_LINIEN_ID", "FK_HALTESTELLEN_ID"]) \
    .set_index("STEIG_ID", drop=False)

display(stations.head())
display(linien.head())
display(steige.head())

,HALTESTELLEN_ID,NAME,GEMEINDE_ID,LAT,LNG
HALTESTELLEN_ID,,,,,
214460106,214460106,Schrankenberggasse,90001,48.173801,16.389807
214460107,214460107,Achengasse,90001,48.284526,16.448898
214460108,214460108,Ada-Christen-Gasse,90001,48.152866,16.385954
214460109,214460109,Adam-Betz-Gasse,90001,48.215611,16.535191
214460110,214460110,Adamovichgasse,90001,48.142167,16.337840


,LINIEN_ID,BEZEICHNUNG,VERKEHRSMITTEL
LINIEN_ID,,,
214433717,214433717,D,ptTram
406201771,406201771,N17,Pt_RufbusNacht
214433953,214433953,N20,ptBusNight
214433955,214433955,N23,ptBusNight
406201845,406201845,N24,Pt_RufbusNacht


,STEIG_ID,FK_LINIEN_ID,FK_HALTESTELLEN_ID
STEIG_ID,,,
214689748,214689748,214433691,214461074
214689749,214689749,214433691,214461382
214689750,214689750,214433691,214461121
214689751,214689751,214433691,214460711
214689752,214689752,214433691,214461278


## Bearbeiten des Datasets

### Stationen der U Bahnen

Erstellen Sie einen Dataframe *metroStations*. Er soll nur die Haltestellen beinhalten, die von einer
U Bahn (Verkehrsmittel ist *ptMetro*) angefahren werden. Gehe dabei so vor: Erstelle einen Dataframe
*metros* mit den U Bahn Liniene. Das ist ein einfacher Filter, wo das Verkehrsmittel von *linien*
auf den Wert *ptMetro* verglichen wird.

Erstellen Sie danach einen Dataframe *metroSteige* für alle Steige, wo eine U Bahn abfährt. Verwenden Sie dafür
einen Filter mit *isin*. Er kann so verwendet werden: *df[df.COLUMN.isin(other_df.COLUMN)]*. *df*
steht für einen Dataframe. Es wird geprüft, ob der Wert von *df.COLUMN* in der Liste von *other_df.COLUMN*
vorkommt.

Zum Schluss erstellen Sie den Dataframe *metroStations*. Hier kann wieder *isin()* verwendet werden. Es
gibt 96 Stationen mit einer U Bahn.

In [84]:
# TODO: Write your code.

### Join mit den Adressen

Im Dataframe *addresses* liegen alle Adressen von Wien. Diese muss mit dem Dataframe *metroStations*
so verknüpft werden, dass jede Adresse mit jeder Station kombiniert wird. Dies ist ein sogenannter
*Cross Join*. Die Methode *join()* besitzt einen Parameter *how=cross*, der das ermöglicht. Auf 
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html
sind Informationen dazu. Vergiss nicht, das Suffix zu setzen, da die Spalten LAT und LNG in beiden
Dataframes vorkommen. Schreibe das Ergebnis in den Dataframe *addressWithStations*

Das Ergebnis sind 14352000 Zeilen (!)

In [85]:
# TODO: Write your code.

### Berechnen der Entfernungen

In [ ]:
import math
def calcDistance(lat, lng, lat_dest, lng_dest):
    """
    Berechnet die näherungsweise Entfernung zwischen 2 Koodinaten. Dabei wird aus Performancegründen
    eine Kugelgestalt der Erde angenommen.
    :lat, lng, lat_dest, lng_dest: Koordinaten in Grad.
    :return: Entfernung in Meter.
    """
    rad=math.pi/180
    return math.acos(math.sin(lat_dest*rad) * math.sin(lat*rad) + math.cos(lat_dest*rad) * math.cos(lat*rad) * math.cos((lng-lng_dest)*rad)) * 6371000

Erstellen Sie nun eine Spalte *DIST* im Dataframe *addressWithStations*. Mit *apply* rufen Sie für
jede Zeile die Funktion *calcDistance* auf. Lassen Sie die Berechnung laufen. Falls Ihnen nach einigen
Minuten langweilig wird: Sie werden das Ergebnis vermutlich auch nach einigen Stunden nicht bekommen.
Wir haben schließlich 14 Millionen Datensätze.

Aber ist das eigentlich nötig? In der Statistik wird oft mit Stichproben gearbeitet, um ein Gesamtbild
mit guter Genauigkeit zu erhalten. Jetzt kommt die Funktion *sample()* ins Spiel. Sie können z. B.
mit *df.sample(1000)* 1000 zufällige Werte aus einem Dataframe lesen. Die Zufälligkeit ist für die
Stichprobe wichtig.

Ändern Sie nun den Join in der vorigen Zelle, sodass nicht *addresses* mit *metroStations* verknüpft
wird, sondern ein Sample von 10 000 Adressen mit den Stationen. addressWithStations beinhaltet also
*10 000 x Anzahl der U Bahn Stationen* Datensätze.

In [86]:
# TODO: Write your code.

## Aggregieren der Information

Der Dataframe *addressWithStations* hat nun eine Entfernungsabgabe zu jeder U Bahn Station. Wir wollen
die nächste Station berücksichtigen. Das ist die minimale Entfernung von der Adresse aus. Erstellen
Sie einen Dataframe *distanceToNearest*, der die Adress ID (ADRCD), die PLZ und die minimale Distanz
beinhaltet. Löschen Sie den entstandenen Index, damit Sie wieder auf die Spalten zugriefen können.

In [87]:
# TODO: Write your code.

Fügen Sie nun eine Spalte *IS_REACHABLE* zum Dataframe *distanceToNearest* hinzu. Die Spalte ist 1,
wenn DIST kleiner als 1000m ist. Sonst ist die Spalte 0.

In [88]:
# TODO: Write your code.

Für die endgültige Statistik gruppieren Sie nun nach der PLZ und aggregieren Sie die Spalte
*IS_REACHABLE* mit der Summe und der Anzahl (size). Erstellen Sie danach eine Spalte *PERCENT*, die
den Prozentwert der Adressen dieser PLZ angibt, die eine nahe Station haben. Mit *round(2)* können Sie
das Ergebnis der Berechnung auf 2 Stellen runden.

In [89]:
# TODO: Write your code.